In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import GPy
from diGP.preprocessing import (readHCP,
                                averageb0Volumes,
                                createBrainMaskFromb0Data,
                                replaceNegativeData)
from diGP.dataManipulations import (DataHandler,
                                    log_q_squared,
                                    generateCoordinates)
from diGP.generateSyntheticData import combineCoordinatesAndqVecs

%matplotlib inline

 C:\Users\sesjojen\AppData\Local\Continuum\Anaconda3\lib\site-packages\skimage\filter\__init__.py:6: skimage_deprecation:The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.


In [4]:
dataPath = 'C:\\Users\\sesjojen\\Documents\\Data\\HumanConnectomeProject\\mgh_1007\\diff\\preproc'
print(dataPath)

C:\Users\sesjojen\Documents\Data\HumanConnectomeProject\mgh_1007\diff\preproc


In [5]:
gtab, data, voxelSize = readHCP(dataPath)
print(gtab.info)

B-values shape (552,)
         min 0.000000 
         max 10050.000000 
B-vectors shape (552, 3)
         min -0.998755 
         max 0.998348 
None


In [6]:
data = replaceNegativeData(data, gtab)

b0 = averageb0Volumes(data, gtab)
mask = createBrainMaskFromb0Data(b0)
maskIdx = np.nonzero(mask)

In [7]:
def normalize_signal(data, maskIdx, b0):
    S = data[maskIdx[0], maskIdx[1], maskIdx[2], :]
    S = S/b0[maskIdx[0], maskIdx[1], maskIdx[2], np.newaxis]
    maxSignal = 1.5
    percentExceedingMaxSignal = 100*np.sum(S>maxSignal)/np.prod(S.shape)
    print('Replacing the top {} % values with {}.'.format(percentExceedingMaxSignal, maxSignal))
    S[S > maxSignal] = maxSignal
    return S

S = normalize_signal(data, maskIdx, b0)

Replacing the top 0.11240637157867965 % values with 1.5.


Just to detect problems further down the road, we will for now reduce the spatial extent used.

In [8]:
handler = DataHandler(gtab, data[60:70, 60:70, 35:45, :] + 1e-6, voxelSize=voxelSize, spatialIdx=None,
                 box_cox_lambda=1., qMagnitudeTransform=log_q_squared)

In [9]:
X = combineCoordinatesAndqVecs(handler.X_coordinates, handler.X_q)
y = handler.y.reshape(-1, 1)

In [10]:
scaling = 0.3
spatialLengthScale = 5
bValLengthScale = 3

kernel = (GPy.kern.RBF(input_dim=1, active_dims=[0],
                       variance=1,
                       lengthscale=spatialLengthScale) *
          GPy.kern.RBF(input_dim=1, active_dims=[1],
                       variance=1,
                       lengthscale=spatialLengthScale) *
          GPy.kern.RBF(input_dim=1, active_dims=[2],
                       variance=1,
                       lengthscale=spatialLengthScale) *
          GPy.kern.RBF(input_dim=1, active_dims=[3],
                       variance=1,
                       lengthscale=bValLengthScale) *
          GPy.kern.LegendrePolynomial(
             input_dim=3,
             coefficients=np.array((2, 0.5, 0.05)),
             orders=(0, 2, 4),
             active_dims=(4, 5, 6)))

kernel.parts[0].variance.fix(value=1)
kernel.parts[1].variance.fix(value=1)
kernel.parts[2].variance.fix(value=1)
kernel.parts[3].variance.fix(value=1)

array([0])

In [12]:
print(kernel)

  mul.                             |  value  |  constraints  |  priors
  rbf.variance                     |    1.0  |   fixed +ve   |        
  rbf.lengthscale                  |    5.0  |      +ve      |        
  rbf_1.variance                   |    1.0  |   fixed +ve   |        
  rbf_1.lengthscale                |    5.0  |      +ve      |        
  rbf_2.variance                   |    1.0  |   fixed +ve   |        
  rbf_2.lengthscale                |    5.0  |      +ve      |        
  rbf_3.variance                   |    1.0  |   fixed +ve   |        
  rbf_3.lengthscale                |    3.0  |      +ve      |        
  LegendrePolynomial.coefficients  |   (3,)  |      +ve      |        


In [11]:
grid_dims = [[0], [1], [2], [3, 4, 5, 6]]

model = GPy.models.GPRegressionGrid(X, y, kernel, grid_dims=grid_dims)        

 C:\Users\sesjojen\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:224: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future


ValueError: total size of new array must be unchanged

In [ ]:
%debug

In [ ]:
model.optimize_restarts(messages=self.verbose)